<p style="background-color:#D0D0FF; padding:10px;">This notebook presents solutions to the exercises in <a href="http://www.greenteapress.com/compmod/html/thinkcomplexity004.html" target="_blank">Chapter 3</a> of Allen B. Downey's <a href="http://www.greenteapress.com/compmod/" target="_blank">Think Complexity</a></p>

In [1]:
# provided functions
class LinearMap(object):

    def __init__(self):
        self.items = []

    def add(self, k, v):
        self.items.append((k, v))

    def get(self, k):
        for key, val in self.items:
            if key == k:
                return val
        raise KeyError
        
class BetterMap(object):

    def __init__(self, n=100):
        self.maps = []
        for i in range(n):
            self.maps.append(LinearMap())

    def find_map(self, k):
        index = hash(k) % len(self.maps)
        return self.maps[index]

    def add(self, k, v):
        m = self.find_map(k)
        m.add(k, v)

    def get(self, k):
        m = self.find_map(k)
        return m.get(k)
    
class HashMap(object):

    def __init__(self):
        self.maps = BetterMap(2)
        self.num = 0

    def get(self, k):
        return self.maps.get(k)

    def add(self, k, v):
        if self.num == len(self.maps.maps):
            self.resize()

        self.maps.add(k, v)
        self.num += 1

    def resize(self):
        new_maps = BetterMap(self.num * 2)

        for m in self.maps.maps:
            for k, v in m.items:
                new_maps.add(k, v)

        self.maps = new_maps
        
lm = LinearMap()
bm = BetterMap(2)
hm = HashMap()
for f in (lm,bm,hm):
    print(f)

<p style="background-color:#D0D0FF; padding:10px;"><em>Exercise 3:</em> Write a function called bisection that takes a sorted list and a target value and returns the index of the value in the list, if it’s there, or None if it’s not. Or you could read the documentation of the bisect module ([link](https://docs.python.org/3/library/bisect.html?highlight=bisect#module-bisect)) and use that!</p>

In [2]:
from bisect import bisect
big_list = range(1000)

print(bisect(big_list, 17))
print(big_list.index(17)+1)

18
18


**Note:** Using bisect was more efficient in Python 2, but lists are one of many performance improvements in Python 3

In [3]:
idx = 17
%timeit bisect(big_list, idx)
%timeit big_list.index(idx)

1000000 loops, best of 3: 1.51 µs per loop
The slowest run took 5.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 355 ns per loop


In [4]:
idx = 500
%timeit bisect(big_list, idx)
%timeit big_list.index(idx)

100000 loops, best of 3: 2.1 µs per loop
The slowest run took 4.90 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 381 ns per loop


In [5]:
idx = 947
%timeit bisect(big_list, idx)
%timeit big_list.index(idx)

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.08 µs per loop
The slowest run took 6.16 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 379 ns per loop


<p style="background-color:#D0D0FF; padding:10px;"><em>Exercise 4:</em> My implementation of HashMap accesses the attributes of BetterMap directly, which shows poor object-oriented design.
    <br /><br />
    1: The special method __len__ is invoked by the built-in function len. Write a __len__ method for BetterMap and use it in add.
    <br /><br />
    2: Use a generator to write BetterMap.iteritems, and use it in resize.</p>

In [6]:
# Part 1 # Add __len__ function to BetterMap
def BetterMap__len__(self):
    '''Return length of BetterMap instance'''
    return len(self.maps)
BetterMap.__len__ = BetterMap__len__

BetterMap(2).maps, len(BetterMap(2))

([<__main__.LinearMap at 0x14957212588>,
 2)

In [7]:
# Part 1 # Update add function to HashMap
def HashMapadd(self, k, v):
    '''Add key/value pair to HashMap'''
    if self.num == len(self.maps):
        self.resize()

    self.maps.add(k, v)
    self.num += 1
    
HashMap.add = HashMapadd

h = HashMap()
h.add('a',1)
h.get('a')

1

In [8]:
# Part 2 # Use a generator to write BetterMap.iteritems
def BetterMapiteritems(self):
    '''Iterate through key/value pairs in BetterMap'''
    for m in self.maps:
        for k, v in m.items:
            yield (k,v)

BetterMap.iteritems = BetterMapiteritems

bm = BetterMap(2)
bm.add('a',1)
for k, v in bm.iteritems():
    print(k,v)

a 1


In [9]:
# Part 2 # Use BetterMap.iteritems in HashMap.resize.
def HashMapresize(self):
    '''Resize maps to make room for new values'''
    new_maps = BetterMap(self.num * 2)

    for k, v in self.maps.iteritems():
        new_maps.add(k, v)

    self.maps = new_maps

HashMap.resize = HashMapresize

h.resize()

<p style="background-color:#D0D0FF; padding:10px;"><em>Exercise 5:</em> A drawbacks of hashtables is that the elements have to be hashable, which usually means they have to be immutable. That’s why, in Python, you can use tuples but not lists as keys in a dictionary. An alternative is to use a tree-based map.
    <br /><br />
    Write an implementation of the map interface called TreeMap that uses a <a href="http://en.wikipedia.org/wiki/Red-black_tree" target="_blank">red-black tree</a> to perform add and get in log time. </p>

This is a solved problem. A work in progress is shown below, but refer to [this reference](http://www.cs.auckland.ac.nz/software/AlgAnim/red_black.html), [the bintrees package](https://pypi.python.org/pypi/bintrees/), and [this other reference](http://eternallyconfuzzled.com/tuts/datastructures/jsw_tut_rbtree.aspx)

In [10]:
class RedBlackInt(int):
    def __init__(self, key, value=None, parent=None, left_child=None, right_child=None):
        int.__init__(value)
        self.k = key
        if value == None:
            self.v = value
        else:
            self.v = key
        self.p = parent
        self.l = left_child
        self.r = right_child
        self.black = False
    
    def G(self):
        '''Return grandparent'''
        if self.p == None:
            return None
        else:
            return self.p.p
    
    def U(self):
        '''Return uncle'''
        g = self.G()
        
        if g == None:
            return None
        elif self.p == g.l:
            return g.r
        else:
            return g.l

    def __str__(self):
        return str(self.k)

    def __repr__(self):
        return ' -> '.join([ repr(self.p), str(self), '({0:s},{1:s})'.format(repr(self.l), repr(self.r)) ])

    
def RedBlackTree(object):
    def __init__(self, root=None, node_type=RedBlackInt):
        self.root = root
        self.node_type = node_type
        
    def _binary_tree_insert(self, key, value=None):
        '''Recursive function to navigate red-black tree and create new node'''
        RedBlack = type(self)
        if key < self.k:
            if self.l == None:
                node = RedBlack(key, value, self)
                self.l = node
                return node
            else:
                self.l._binary_tree_insert(key, value)
        elif key > self.k:
            if self.r == None:
                node = RedBlack(key, value, self)
                self.r = node
                return node
            else:
                self.r._binary_tree_insert(key, value)
                
    def append(self, key, value=None):
        if value == None:
            value = key
            
        if root == None:
            root = node_type(k, v)
        else: 
            # Insertion begins by adding the node as any binary search tree insertion does and by coloring it red
            self._binary_tree_insert(key, value)
        
        # Case 1: The current node N is at the root of the tree
            
RedBlack = RedBlackInt

n = RedBlack(5)
n+1, str(n), n.G(), repr(n)

(6, '5', None, 'None -> 5 -> (None,None)')

<p style="background-color:#D0D0FF; padding:10px;"><em>Exercise 6:</em> Test the performance of LinearMap, BetterMap and HashMap; see if you can characterize their order of growth.
<br /><br />
You can download my map implementations from thinkcomplex.com/Map.py, and the code I used in this section from thinkcomplex.com/listsum.py.
<br /><br />
You will have to find a range of n that is big enough to show asymptotic behavior, but small enough to run quickly.</p>

In [11]:
def addValues(m, n=10):
    for i in range(n):
        m.add(str(i),i)

In [12]:
for m in [ LinearMap(), BetterMap(2), HashMap() ]:
    print(u'\n',m)
    for n in range(10,40+10,10):
        %timeit addValues(m, n)


The slowest run took 4.52 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.92 µs per loop
100000 loops, best of 3: 13.1 µs per loop
100000 loops, best of 3: 19 µs per loop
10000 loops, best of 3: 23.7 µs per loop

100000 loops, best of 3: 14.3 µs per loop
10000 loops, best of 3: 28.2 µs per loop
10000 loops, best of 3: 41.1 µs per loop
10000 loops, best of 3: 57.2 µs per loop

The slowest run took 4.27 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 24.6 µs per loop
The slowest run took 7.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 46.9 µs per loop
The slowest run took 10.52 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 70.7 µs per loop
10000 loops, best of 3: 94.6 µs per loop


Compare performance against Python's standard dictionary type

In [14]:
def addValues2(d, n=10):
    for i in range(n):
        d[str(i)] = i

m = {}
for n in range(10,40+10,10):
        %timeit addValues2(m, n)

100000 loops, best of 3: 4.18 µs per loop
100000 loops, best of 3: 7.91 µs per loop
100000 loops, best of 3: 11.5 µs per loop
100000 loops, best of 3: 15.3 µs per loop
